# 시간 가중 벡터저장소 리트리버(TimeWeightedVectorStoreRetriever)

`TimeWeightedVectorStoreRetriever` 는 의미론적 유사성과 시간에 따른 감쇠를 결합해 사용하는 검색 도구입니다. 이를 통해 문서 또는 데이터의 **"신선함"** 과 **"관련성"** 을 모두 고려하여 결과를 제공합니다.

스코어링 알고리즘은 다음과 같이 구성됩니다

$\text{semantic\_similarity} + (1.0 - \text{decay\_rate})^{hours\_passed}$

여기서 `semantic_similarity` 는 문서 또는 데이터 간의 의미적 유사도를 나타내고, `decay_rate` 는 시간이 지남에 따라 점수가 얼마나 감소하는지를 나타내는 비율입니다. `hours_passed` 는 객체가 마지막으로 접근된 후부터 현재까지 경과한 시간(시간 단위)을 의미합니다.

이 방식의 주요 특징은, 객체가 마지막으로 접근된 시간을 기준으로 하여 **"정보의 신선함"** 을 평가한다는 점입니다. 즉, **자주 접근되는 객체는 시간이 지나도 높은 점수**를 유지하며, 이를 통해 **자주 사용되거나 중요하게 여겨지는 정보가 검색 결과 상위에 위치할 가능성이 높아집니다.** 이런 방식은 최신성과 관련성을 모두 고려하는 동적인 검색 결과를 제공합니다.

특히, `decay_rate`는 리트리버의 객체가 생성된 이후가 아니라 \*\*마지막으로 액세스된 이후 경과된 시간을 의미합니다. 즉, 자주 액세스하는 객체는 '최신'으로 유지됩니다.


패키지를 업그레이드 합니다.


In [1]:
%pip install -qU deeplake lark

Note: you may need to restart the kernel to use updated packages.


필요한 모듈을 import 합니다.


In [2]:
from datetime import datetime, timedelta

import faiss
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

## 낮은 감쇠율(low decay_rate)

- `decay rate` 가 낮다는 것은 (여기서는 극단적으로 0에 가깝게 설정할 것입니다) **기억이 더 오래 "기억될"** 것임을 의미합니다.

- `decay rate` 가 **0 이라는 것은 기억이 절대 잊혀지지 않는다**는 것을 의미하며, 이는 이 retriever를 vector lookup과 동등하게 만듭니다.


`TimeWeightedVectorStoreRetriever`를 초기화하며, 벡터 저장소, 감쇠율(`decay_rate`)을 매우 작은 값으로 설정하고, 검색할 벡터의 개수(k)를 1로 지정합니다.


In [3]:
# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings()
# 벡터 저장소를 빈 상태로 초기화합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
# 시간 가중치가 적용된 벡터 저장소 검색기를 초기화합니다.
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.0000000000000000000000001, k=1
)

`retriever.add_documents()` 메서드를 사용하여 "hello world" 문자열을 가진 `Document` 객체를 추가하며, 메타데이터로 `last_accessed_at`에 어제 시간을 설정합니다.

다시 `retriever.add_documents()` 메서드를 사용하여 "hello foo" 문자열을 가진 `Document` 객체를 추가합니다.


In [4]:
yesterday = datetime.now() - timedelta(days=1)  # 어제 날짜를 계산합니다.
retriever.add_documents(
    # "hello world" 내용의 문서를 추가하고, 메타데이터에 어제 날짜를 설정합니다.
    [Document(page_content="hello world", metadata={
              "last_accessed_at": yesterday})]
)
# "hello foo" 내용의 문서를 추가합니다.
retriever.add_documents([Document(page_content="hello foo")])

['d3c2b942-2fa2-4d78-9c87-931fbbc7646d']

`retriever.get_relevant_documents("hello world")` 메서드를 호출하여 "hello world"와 관련된 문서를 검색합니다.

- "Hello World"가 가장 먼저 반환되는데, 이는 가장 두드러진(salient) 문서이기 때문입니다.
- `decay_rate` 가 **0에 가깝기 때문** 에 "Hello World" 문서는 여전히 최신(recent)으로 간주됩니다.


In [5]:
# "Hello World"가 가장 먼저 반환되는 이유는 가장 두드러지기 때문이며, 감쇠율이 0에 가깝기 때문에 여전히 최신 상태를 유지하고 있음을 의미합니다.
retriever.get_relevant_documents("hello world")

[Document(page_content='hello world', metadata={'last_accessed_at': datetime.datetime(2024, 3, 23, 0, 25, 31, 137309), 'created_at': datetime.datetime(2024, 3, 23, 0, 25, 30, 544821), 'buffer_idx': 0})]

## 높음 감쇠율(high decay_rate)

높은 `decay_rate`(예: 0.9999...)를 사용하면 `recency score`가 빠르게 0으로 수렴합니다.

(만약 이 값을 1로 설정하면 모든 객체의 `recency` 값이 0이 되어, Vector Lookup 과 동일한 결과를 얻게 됩니다.)


`TimeWeightedVectorStoreRetriever`를 사용하여 검색기를 초기화합니다. `decay_rate`를 0.999로 설정하여 시간에 따른 가중치 감소율을 조정합니다.


In [6]:
# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings()
# 벡터 저장소를 빈 상태로 초기화합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
# 시간 가중치가 적용된 벡터 저장소 검색기를 초기화합니다.
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.999, k=1
)

`retriever.add_documents()` 메서드를 사용하여 "hello world" 문자열을 가진 `Document` 객체를 추가하며, 메타데이터로 `last_accessed_at`에 어제 시간을 설정합니다.

다시 `retriever.add_documents()` 메서드를 사용하여 "hello foo" 문자열을 가진 `Document` 객체를 추가합니다.


In [7]:
yesterday = datetime.now() - timedelta(days=1)  # 어제 날짜를 계산합니다.
retriever.add_documents(
    # "hello world" 내용의 문서를 추가하고, 메타데이터에 어제 날짜를 설정합니다.
    [Document(page_content="hello world", metadata={
              "last_accessed_at": yesterday})]
)
# "hello foo" 내용의 문서를 추가합니다.
retriever.add_documents([Document(page_content="hello foo")])

['076d7c8c-5ee3-4316-b201-55140bbb686f']

- `retriever.get_relevant_documents("hello world")` 를 호출하면 "Hello Foo"가 먼저 반환됩니다.
- 이는 retriever가 "hello world"와 관련된 문서를 대부분 잊어버렸기 때문입니다.


In [8]:
# "hello world"와 가장 관련된 문서를 검색합니다.
# "Hello Foo"가 먼저 반환되는 이유는 "hello world"가 대부분 잊혀졌기 때문입니다.
retriever.get_relevant_documents("hello world")

[Document(page_content='hello foo', metadata={'last_accessed_at': datetime.datetime(2024, 3, 23, 0, 25, 31, 972916), 'created_at': datetime.datetime(2024, 3, 23, 0, 25, 31, 718842), 'buffer_idx': 1})]

## 감쇠율(decay_rate) 정리

- `decay_rate` 를 0.000001 로 매우 작게 설정한 경우: 감쇠율(즉, 정보를 망각하는 비율)이 매우 낮기 때문에 정보를 거의 잊지 않습니다. 따라서, **최신 정보이든 오래된 정보든 시간 가중치 차이가 거의 없습니다.** 이럴때는 유사도에 더 높은 점수를 주게 됩니다.
- `decay_rate` 를 0.999 로 1에 가깝게 설정한 경우: 감쇠율(즉, 정보를 망각하는 비율)이 매우 높습니다. 따라서, 과거의 정보는 거의다 잊어버립니다. 따라서, 이러한 경우는 최신 정보에 더 높은 점수를 주게 됩니다.


## 가상의 시간(Virtual time)

LangChain의 일부 유틸리티를 사용하면 시간 구성 요소를 모의(mock) 테스트 할 수 있습니다.


`mock_now` 함수는 LangChain에서 제공하는 유틸리티 함수로, 현재 시간을 모의(mock)하는 데 사용됩니다.

- 아래는 시간을 2023년 6월 8일로 설정해 보겠습니다.


In [9]:
import datetime

from langchain.utils import mock_now

# 현재 시간을 특정 시점으로 설정
mock_now(datetime.datetime(2023, 6, 8, 0, 0, 0))

# 현재 시간 출력
print(datetime.datetime.now())

2024-03-23 00:25:32.221079


`mock_now` 함수를 사용하여 현재 시간을 2024년 3월 28일 10시 11분으로 설정합니다.

- `retriever.get_relevant_documents` 메서드를 호출하여 "hello world"라는 쿼리에 대한 관련 문서를 검색합니다.
- 검색 결과를 출력합니다. 이때 문서의 마지막 접근 시간이 설정된 시간(2024년 3월 28일 10시 11분)으로 표시됩니다.


In [10]:
# 현재 시간을 2024년 6월 3일 10시 11분으로 설정합니다.
with mock_now(datetime.datetime(2024, 3, 28, 10, 11)):
    # "hello world"와 관련된 문서를 검색하고 출력합니다.
    print(retriever.get_relevant_documents("hello world"))

[Document(page_content='hello world', metadata={'last_accessed_at': MockDateTime(2024, 3, 28, 10, 11), 'created_at': datetime.datetime(2024, 3, 23, 0, 25, 31, 454121), 'buffer_idx': 0})]
